In [1]:
import json
import numpy as np
import yfinance as yf
import random
random.seed(0)
np.random.seed(0)
from pytickersymbols import PyTickerSymbols

def compute_qubits(stock_metadata, budget):
    n_qubits = 0
    for stock in stock_metadata:
        price = stock_metadata[stock]["price"]
        n_qubits += max(1, int(np.ceil(np.log2(np.ceil(budget/price)))))
    return n_qubits

def is_valid_ticker(ticker):
    """Check if a ticker exists by retrieving 1 day of historical data."""
    try:
        data = yf.Ticker(ticker).history(period="1d")
        return not data.empty  # Valid if data is not empty
    except:
        return False  # Invalid ticker
    

def randomly_assign_qubits(n_qubits, stocks):
    """
    Randomly assign qubits to stocks, ensuring each stock gets at least one qubit.
    
    Args:
        n_qubits: Total number of qubits
        stocks: List of stock tickers
        
    Returns:
        qubits_per_stock: Dictionary with number of qubits per stock
    """
    n_stocks = len(stocks)
    if n_qubits < n_stocks and n_qubits != n_stocks:
        raise ValueError("Number of qubits must be at least equal to number of stocks")
    
    # Start by giving each stock one qubit
    qubits_per_stock = {stock: 1 for stock in stocks}
    
    # Randomly distribute remaining qubits
    remaining_qubits = n_qubits - n_stocks
    
    # Use random.choices with weights to distribute remaining qubits
    for _ in range(remaining_qubits):
        # Randomly select a stock to give an additional qubit
        chosen_stock = np.random.choice(stocks)
        qubits_per_stock[chosen_stock] += 1
    
    return qubits_per_stock

stock_data = PyTickerSymbols()
stocks = list(stock_data.get_dow_jones_nyc_yahoo_tickers())
stocks = [s for s in stocks if is_valid_ticker(s)]
print("Available stocks: ", stocks)

experiment_data = []

for n_stocks in range(2, 11):
    print("Running experiment with ", n_stocks, " stocks")
    failed_attempts = 0
    successful_attempts = 0
    while True:

        if successful_attempts > 10:
            print("Successfully ran 10 experiments, moving to next configuration")
            break
        
        if failed_attempts > 100*n_stocks:
            print(f"Failed {100*n_stocks} attempts, skipping to next configuration")
            break

        sampled_stocks = random.sample(stocks, n_stocks)

        start = "2015-01-01"
        end = "2025-01-01"
        data = yf.download(sampled_stocks, start=start, end=end, progress=False)
        prices_now = data["Close"].iloc[-1]

        budget = random.randint(int(np.ceil(max(prices_now))), 6000)

        max_qubits = compute_qubits({stock: {"price": prices_now[stock]} for stock in sampled_stocks}, budget)

        if max_qubits > 15:
            #print("Skipping experiment because max qubits is too high")
            failed_attempts += 1
            continue

        print("Budget: ", budget)
        
        data_point = {}
        data_point["max_qubits"] = max_qubits
        data_point["budget"] = budget
        data_point["stocks"] = sampled_stocks
        data_point["start"] = start
        data_point["end"] = end
        data_point["n_stocks"] = n_stocks
    
        experiment_data.append(data_point)
        successful_attempts += 1

$GS-PK: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Available stocks:  ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'NKE', 'MRK', 'MSFT', 'PG', 'TRV', 'UNH', 'VZ', 'V', 'WMT', 'WBA', 'DIS', 'AMGN', 'HON', 'CRM', 'DOW']
Running experiment with  2  stocks
YF.download() has changed argument auto_adjust default to True
Budget:  3670
Budget:  4758
Budget:  2628
Budget:  4855
Budget:  2124
Budget:  2670
Budget:  1138
Budget:  2292
Budget:  5351
Budget:  2902
Budget:  781
Successfully ran 10 experiments, moving to next configuration
Running experiment with  3  stocks
Budget:  3039
Budget:  3319
Budget:  5486
Budget:  2422
Budget:  4547
Budget:  2257
Budget:  831
Budget:  4450
Budget:  1443
Budget:  4847
Budget:  1137
Successfully ran 10 experiments, moving to next configuration
Running experiment with  4  stocks
Budget:  2063
Budget:  1121
Budget:  2154
Budget:  1812
Budget:  816
Budget:  1143
Budget:  397
Budget:  1861
Budget:  653
Budget:  1573
Budget:  335
Successfully ran 10 exper

In [2]:
experiment_data = sorted(experiment_data, key=lambda x: x["max_qubits"])

with open("experiments_data.json", "w") as f:
    json.dump({"data": experiment_data}, f, indent=4)

In [3]:
print(len(experiment_data), " experiments generated")

73  experiments generated
